# 16. Linear Data Stuctures

## Lists
---

### The "List" Abstract Data Structure 

#### What is an **Abstract Data Type** (**ADT**)

In general, **abstract data types** (**ADT**) gives us a **definition** (**abstraction**) of the specific structure, i.e. **defines the allowed operations and properties**, without being interested in the specific implementation. This allows an **abstract data type** to have several different implementations and respectively different efficiency.

<br>

#### Introducing the `System.Collections.IList` Interface

Interfaces in C# construct a frame (contract) for their implementations – classes.    
   
This contract consists of **a set of methods and properties**, which
each class must implement in order to implement the interface.

Each **ADT** defines some interface. Let’s consider the interface `System.Collections.IList`.     
The **basic methods** which it defines are:

- `int Add(object)` – adds element in the end of the list 
- `void Insert(int, object)` – adds element on a chosen position in the list
- `void Clear()` – removes all elements in the list 
- `bool Contains(object)` – checks whether the list contains the element 
- `void Remove(object)` – removes the element from the list 
- `void RemoveAt(int)` – removes the element on a given position 
- `int IndexOf(object)` – returns the position of the element 
- `this[int]` – indexer, allows access to the elements on a set position

<br>

<br>

### Static Lists (An Array-Based Implementation)

##### Lists v.s. Arrays

Somewhat similarly to <a href="../07. Arrays/The Anatomy of an Array.ipynb">Arrays</a>, we could imagine the **list** as an **ordered sequence (line) of elements**.

A List features a length property (count of elements), and though it's elements are arranged consecutively, their addresses in memory are, unlike Arrays, non-contiguous (they don't live right next to each other).

Notwithsatnding, **Arrays perform many of the functions of the list ADT**, but there is an even more significant difference to consider – that Lists allow adding new elements, while **Arrays have fixed capacity**.

<br>

##### The Resizable Array-based List

Even so, there exists implementation techniques for an Array what make it possible to **automatically resize it's capacity** (in a similar fashion to the [StringBuilder](../13.%20Strings%20and%20Text%20Processing/Building%20Strings.ipynb) class).   

Such a list may be reffered to as a **Static List implemented with an extensible Array**.

Using [Generics](../14.%20Defining%20Classes/Generics.ipynb), we may define such an Array-based structure able to hold any type of data, as defined below:

In [1]:
public class ResizableArray<T>
{

    // FIELD DEFINITIONS --------------------------------------------------
    private T[]       array;
    private int       occupiedElements;
    private const int DEFAULT_CAPACITY = 4;
    //---------------------------------------------------------------------


    // PROPERTY DEFINITIONS -----------------------------------------------
    public int OccupiedElements
    {
        // Read Only
        get { return this.occupiedElements; }
    }
    //---------------------------------------------------------------------
    public T[] ArrayState
    {
        // Read Only
        get { return this.array; }
    }
    //---------------------------------------------------------------------
    //------------------------- I N D E X E R -----------------------------
    public T this[ int index ]
    {

        get
        {

            // Try to assign the given value at the specified index,
            // proceeding cautiously in the event that the given index
            // could possibly be out of the boundaries of the Array
            try
            {
                return this.array[ index ];
            }


            // Should there be an error, 
            // it's sure to be that the specified index is out of range.
            // so provide an appropriate exception which generates a 
            // message to the user, 
            // and return the given type's default equivalent to null
            // in order to indicate that the operation has failed
            catch ( IndexOutOfRangeException ioor )
            {
                Console.WriteLine(
                    $"Index: {index} is invalid\n\n"
                    +
                    $"Stack Trace: {ioor.StackTrace}"
                );

                return default(T);
            }

        }
 

        set
        {

            // Try to assign the given value at the specified index,
            // proceeding cautiously in the event that the given index
            // could possibly be out of the boundaries of the Array
            try
            {
                this.array[ index ] = value;
            }


            // Should there be an error, 
            // it's sure to be that the specified index is out of range.
            // so provide an appropriate exception which generates a 
            // message to the user
            catch ( IndexOutOfRangeException ioor )
            {
                Console.WriteLine(
                    $"Index: {index} is invalid\n\n"
                    +
                    $"Stack Trace: {ioor.StackTrace}"
                );
            }

        }
    }
    //---------------------------------------------------------------------


    // CONSTRUCTOR --------------------------------------------------------
    public ResizableArray( int capacity = DEFAULT_CAPACITY )
    {
        this.array = new T[ capacity ];
        this.occupiedElements = 0;
    }
    //---------------------------------------------------------------------


    // METHODS ------------------------------------------------------------
    //---------------------- I N S E R T I O N ----------------------------
    // Double the size of the array, should it be determined to be full
    public void ResizeIfArrayIsFull()
    {

        // IF the count of occupied elements, 
        // once adusted for zero indexing, 
        // and then subsequently incremented,
        // is found exceed the capacity of the array:
        if( this.OccupiedElements + 1 > this.array.Length )
        {

            // Declare a new array with twice the capacity
            T[] resizedArray = new T[ this.array.Length * 2 ];


            // Copy the occupied Array elements into the resized array
            Array.Copy(this.array, resizedArray, this.OccupiedElements); 


            // Point the Copy's object reference back to the orginal Array
            this.array = resizedArray;

        }

    }
    //---------------------------------------------------------------------
    // Append a new element to the end of the array
    public void Add( T elementToAdd )
    {

        // Double the size of the array, should it be determined to be full
        ResizeIfArrayIsFull();


        // Assign the given argument as the last element of the array,
        // taking care to simultaneously increment 
        // the count of occupied elements
        this.array[ this.occupiedElements++ ] = elementToAdd;

    }
    //---------------------------------------------------------------------
    // Insert a new element in the array at the specified index
    public void Insert( int indexToInsertAt, T elementToInsert )
    {

        // Try to insert the given element using the specified index,
        // proceeding cautiously in the event that the given index
        // could possibly be out of the boundaries contained by the Array
        try
        {

            // Double the size of the array, 
            // should it be determined to be full
            ResizeIfArrayIsFull();


            // Shift the each of the elements after the specified index
            // one position to the right in order to accomodate the space
            // needed to insert the specified element
            Array.Copy(
                this.array, indexToInsertAt,
                this.array, indexToInsertAt + 1,
                this.occupiedElements - indexToInsertAt
            );


            // Clobber the element currently at the specified index
            // with the specified element
            this.array[ indexToInsertAt ] = elementToInsert;


            // Increment the the count of occupied elements
            this.occupiedElements++; 

        }


        // Should there be an error, 
        // it's sure to be that the specified index is out of range.
        // so provide an appropriate exception which generates a 
        // message to the user
        catch ( IndexOutOfRangeException ioor )
        {
            Console.WriteLine(
                $"Index: {indexToInsertAt} is invalid\n\n"
                +
                $"Stack Trace: {ioor.StackTrace}"
            );
        }

    }
    //---------------------------------------------------------------------
    //------------------------ S E A R C H --------------------------------
    // Return the Index of the first occurence of the specified element,
    // or -1, should no such element be found to exist.
    public int IndexOf( T element )
    {

        // Iterating For every occupied element index within the array
        for( int index = 0; index < this.occupiedElements + 1; index++)
        {

            // IF the specified element is determined to
            // point to the same object reference in the heap
            // as does the element discovered at the current index: 
            if( object.Equals( element, this.array[ index ] ) )
            {

                // We have a match,
                // return the index corresponding
                // to the first match occurence
                return index;

            }

        }


        // After all that,
        // if we haven't already returned anything,
        // we can safely presume that no such element 
        // could be found in the array,
        // so return -1 to indicate as such
        return -1;

    }
    //---------------------------------------------------------------------
    // Return a True or False value indicating whether a specified
    // element can be determined to exist in the array
    public bool Contains( T element )
    {

        // Search for the Index belonging to the first occurence 
        // of the specified element and return it if found,
        // or else return -1, should no such element be found to exist.
        // Then, return a True or False value corresponsive to that result
        return ( IndexOf( element ) != 1 );
 
    }
    //---------------------------------------------------------------------
    //------------------------- D E L E T I O N ---------------------------
    // Remove and return the element at the specified index
    public T RemoveAt( int indexToRemoveAt )
    {

        // Try to insert the given element using the specified index,
        // proceeding cautiously in the event that the given index
        // could possibly be out of the boundaries contained by the Array
        try
        {

            // Retrieve the element to remove from the specified index
            T elementToRemove = this.array[ indexToRemoveAt ];


            // Shift the each of the elements before the specified index
            // one position to the left in order to account for the gap 
            // left by the element that will be deleted
            Array.Copy(
                this.array, indexToRemoveAt + 1,
                this.array, indexToRemoveAt,
                this.occupiedElements - indexToRemoveAt - 1
            );


            // Assign the given type's default equivalent of a null value 
            // to the last array element,
            // which now holds a duplicate value following the left shift,
            // in order to account for the element's deletion,
            // while simultaneously taking care to decrement the 
            // count of occupied elements accordingly 
            this.array[ (this.occupiedElements--) - 1 ] = default(T);


            // return the deleted element
            return elementToRemove;

        }


        // Should there be an error, 
        // it's sure to be that the specified index is out of range.
        // so provide an appropriate exception which generates a 
        // message to the user, 
        // and return the given type's default equivalent to null
        // in order to indicate that the operation has failed
        catch ( IndexOutOfRangeException ioor )
        {
            Console.WriteLine(

                (indexToRemoveAt == -1) ?
                    $"The element that you specified could not be found\n\n"
                    +
                    $"Stack Trace: {ioor.StackTrace}" 
                    :
                    $"Index: {indexToRemoveAt} is invalid\n\n"
                    +
                    $"Stack Trace: {ioor.StackTrace}"
                    
            );

            return default(T);
        }      
    } 
    //---------------------------------------------------------------------
    // Delete the specified element, 
    // and return the index at which is was found and deleted,
    // or else return a -1 if no such element could be found  
    public int Remove( T element )
    {

        // Observe the matching index at which the specified element
        // was found, provided any such element could be found to exist,
        // otherwise, assign a value of -1 to indicate as such
        int deletedIndex = IndexOf( element );   


        // If the specified element to be deleted
        // can be found in the array:
        if ( Contains( element ) )
        {

            // delete the element specified at the index to delete
            RemoveAt( deletedIndex );

        }


        // return the matching index,
        // or else a -1 to indicate no match could be found
        return deletedIndex; 
 
    }
    //---------------------------------------------------------------------
    // Clear (delete) ALL of the array's elements 
    // and then reset it back to it's default capacity 
    public void Clear()
    {

        // Point the array's object reference 
        // to a newly allocated array sized to a default capacity
        this.array = new T[ DEFAULT_CAPACITY ];


        // Reset the count of occupied elements back to 0
        this.occupiedElements = 0;

    }
    //---------------------------------------------------------------------

}

<br>

So let's now demonstrate some of the `ResizableArray` features that we have defined above.  

First we instantiate a new `ResizableArray` object, which will consist of `int` types for this implementation:

In [2]:
ResizableArray<int> resizableIntArray = new ResizableArray<int>();  

In [3]:
resizableIntArray

OccupiedElements,ArrayState
0,"[ 0, 0, 0, 0 ]"


<br>

let's now populate our `resizableIntArray` instance with consecutive integers ranging from $1$ to $7$:

In [4]:
for( int consecutiveInteger = 1; consecutiveInteger <= 7; consecutiveInteger++ )
{
    resizableIntArray.Add( consecutiveInteger );
}

In [5]:
resizableIntArray

OccupiedElements,ArrayState
7,"[ 1, 2, 3, 4, 5, 6, 7, 0 ]"


Observe that the `resizableIntArray` automatically resized itself to accomodate the full range of additional elements.

<br>

Now that we have a fully populated our `resizableIntArray`, let's now insert the value $99$ at index 3:

In [6]:
resizableIntArray.Insert(3, 99);

<p style="text-align: center;">
    Observe that the <span style="color: blue; font-weight:bold">source index</span> is <span style="color: blue; font-weight:bold">index 3</span> and the <span style="color: red; font-weight:bold">desitation index</span> is <span style="color: red; font-weight:bold">index 4</span> of the same array
</p>

<table style="margin-left: auto; margin-right: auto;">
    <tbody>
        <tr>
            <td style="border: 1px solid black">1</td>
            <td style="border: 1px solid black">2</td>
            <td style="border: 1px solid black">3</td>
            <td style="border: 1px solid black; background: blue; color: white;">4</td>
            <td style="border: 1px solid black; background: red; color: white;">5</td>
            <td style="border: 1px solid black">6</td>
            <td style="border: 1px solid black">7</td>
            <td style="border: 1px solid black">0</td>
        </tr>
        <tr>
            <td style="background: white; border: none;">0</td>
            <td style="background: white; border: none;">1</td>
            <td style="background: white; border: none;">2</td>
            <td style="background: white; font-weight: bold; border: none; color: blue">3</td>
            <td style="background: white; font-weight: bold; border: none; color: red">4</td>
            <td style="background: white; border: none">5</td>
            <td style="background: white; border: none">6</td>
            <td style="background: white; border: none">7</td>
        </tr>
    </tbody>
</table>

$$\Huge{\downarrow}$$

<p style="text-align: center;">
    To perform the insertion, we must first shift the <span style="color: blue; font-weight:bold">element at the source index</span>, in addition to the  <span style="color: red; font-weight:bold"> next 3 consecutive elements</span>, over <span style="font-weight: bold;">one position to their immediate right<span>
</p>

<table style="margin-left: auto; margin-right: auto;">
    <tbody>
        <tr>
            <td style="border: 1px solid black">1</td>
            <td style="border: 1px solid black">2</td>
            <td style="border: 1px solid black">3</td>
            <td style="border: 1px solid black">4</td>
            <td style="border: 1px solid black; background: blue; color: white;">4</td>
            <td style="border: 1px solid black; background: red; color: white;">5</td>
            <td style="border: 1px solid black; background: red; color: white;">6</td>
            <td style="border: 1px solid black; background: red; color: white;">7</td>
        </tr>
        <tr>
            <td style="background: white; border: none;">0</td>
            <td style="background: white; border: none;">1</td>
            <td style="background: white; border: none;">2</td>
            <td style="background: white; border: none;">3</td>
            <td style="background: white; font-weight: bold; border: none;">4</td>
            <td style="background: white; font-weight: bold; border: none;">5</td>
            <td style="background: white; font-weight: bold; border: none;">6</td>
            <td style="background: white; font-weight: bold; border: none;">7</td>
        </tr>
    </tbody>
</table>

$$\Huge{\downarrow}$$

<p style="text-align: center;">
    Finally, we <span style= "font-weight: bold;">clobber</span> the duplicate element remaining at the <span style="color: green; font-weight:bold">specified index</span> with the <span style="color: green; font-weight:bold">specified element value</span>
</p>


<table style="margin-left: auto; margin-right: auto;">
    <tbody>
        <tr>
            <td style="border: 1px solid black;">1</td>
            <td style="border: 1px solid black;">2</td>
            <td style="border: 1px solid black;">3</td>
            <td style="border: 1px solid black; background: green; color: white;">99</td>
            <td style="border: 1px solid black;">4</td>
            <td style="border: 1px solid black;">5</td>
            <td style="border: 1px solid black;">6</td>
            <td style="border: 1px solid black;">7</td>
        </tr>
        <tr>
            <td style="background: white; border: none;">0</td>
            <td style="background: white; border: none;">1</td>
            <td style="background: white; border: none;">2</td>
            <td style="background: white; font-weight: bold; border: none; color: green">3</td>
            <td style="background: white; border: none;">4</td>
            <td style="background: white; border: none;">5</td>
            <td style="background: white; border: none;">6</td>
            <td style="background: white; border: none;">7</td>
        </tr>
    </tbody>
</table>

In [7]:
resizableIntArray

OccupiedElements,ArrayState
8,"[ 1, 2, 3, 99, 4, 5, 6, 7 ]"


<br>

We can now perform search operations to detect the value we just inserted:

In [8]:
// Does the array contain the value?
resizableIntArray.Contains( 99 )

True

In [9]:
// At which Index is that value now located?
resizableIntArray.IndexOf( 99 )

3

<br>

Let's now remove the value which we just inserted: 

In [10]:
resizableIntArray.RemoveAt(3);

<p style="text-align: center;">
    Observe that, in an opposite fashion from the <code>Add()</code> method,<br> the <span style="color: blue; font-weight:bold">source index</span> is <span style="color: blue; font-weight:bold">index 4</span> and the <span style="color: red; font-weight:bold">desitation index</span> is <span style="color: red; font-weight:bold">index 3</span> of the same array
</p>

<table style="margin-left: auto; margin-right: auto;">
    <tbody>
        <tr>
            <td style="border: 1px solid black;">1</td>
            <td style="border: 1px solid black;">2</td>
            <td style="border: 1px solid black;">3</td>
            <td style="border: 1px solid black; background: red; color: white;">99</td>
            <td style="border: 1px solid black; background: blue; color: white;">4</td>
            <td style="border: 1px solid black;">5</td>
            <td style="border: 1px solid black;">6</td>
            <td style="border: 1px solid black;">7</td>
        </tr>
        <tr>
            <td style="background: white; border: none;">0</td>
            <td style="background: white; border: none;">1</td>
            <td style="background: white; border: none;">2</td>
            <td style="background: white; font-weight: bold; border: none;">3</td>
            <td style="background: white; font-weight: bold; border: none;">4</td>
            <td style="background: white; border: none">5</td>
            <td style="background: white; border: none">6</td>
            <td style="background: white; border: none">7</td>
        </tr>
    </tbody>
</table>

$$\Huge{\downarrow}$$

<p style="text-align: center;">
    Once again, in an opposite fashion from the <code>Add()</code> method, to perform the deletion, we must first shift the <span style="color: blue; font-weight:bold">element at the source index</span>, in addition to the  <span style="color: red; font-weight:bold"> next 3 consecutive elements</span>, over <span style="font-weight: bold;">one position to their immediate left<span>
</p>

<table style="margin-left: auto; margin-right: auto;">
    <tbody>
        <tr>
            <td style="border: 1px solid black">1</td>
            <td style="border: 1px solid black">2</td>
            <td style="border: 1px solid black">3</td>
            <td style="border: 1px solid black; background: blue; color: white;">4</td>
            <td style="border: 1px solid black; background: red; color: white;">5</td>
            <td style="border: 1px solid black; background: red; color: white;">6</td>
            <td style="border: 1px solid black; background: red; color: white;">7</td>
            <td style="border: 1px solid black">7</td>
        </tr>
        <tr>
            <td style="background: white; border: none;">0</td>
            <td style="background: white; border: none;">1</td>
            <td style="background: white; border: none;">2</td>
            <td style="background: white; font-weight: bold; border: none;">3</td>
            <td style="background: white; font-weight: bold; border: none;">4</td>
            <td style="background: white; font-weight: bold; border: none;">5</td>
            <td style="background: white; font-weight: bold; border: none;">6</td>
            <td style="background: white; font-weight: bold; border: none;">7</td>
        </tr>
    </tbody>
</table>

$$\Huge{\downarrow}$$

<p style="text-align: center">
    Observe that, following the left shift operation, the last element of the <code>resizableArray</code> now holds a <span style="font-weight: bold; color: goldenrod;">duplicate element</span>
</p>

<table style="margin-left: auto; margin-right: auto;">
    <tbody>
        <tr>
            <td style="border: 1px solid black">1</td>
            <td style="border: 1px solid black">2</td>
            <td style="border: 1px solid black">3</td>
            <td style="border: 1px solid black">4</td>
            <td style="border: 1px solid black">5</td>
            <td style="border: 1px solid black">6</td>
            <td style="border: 1px solid black">7</td>
            <td style="border: 1px solid black; background: goldenrod; color: white;">7</td>
        </tr>
        <tr>
            <td style="background: white; border: none;">0</td>
            <td style="background: white; border: none;">1</td>
            <td style="background: white; border: none;">2</td>
            <td style="background: white; border: none;">3</td>
            <td style="background: white; border: none;">4</td>
            <td style="background: white; border: none;">5</td>
            <td style="background: white; border: none;">6</td>
            <td style="background: white; font-weight: bold; border: none; color: goldenrod">7</td>
        </tr>
    </tbody>
</table>

$$\Huge{\downarrow}$$

<p style="text-align: center;">
    Finally, we <span style= "font-weight: bold;">clobber</span> the duplicate element remaining at the <span style="color: green; font-weight:bold">specified index</span> with the <span style="color: green; font-weight:bold">default empty value corresponding to the given array type</span>
</p>


<table style="margin-left: auto; margin-right: auto;">
    <tbody>
        <tr>
            <td style="border: 1px solid black">1</td>
            <td style="border: 1px solid black">2</td>
            <td style="border: 1px solid black">3</td>
            <td style="border: 1px solid black">4</td>
            <td style="border: 1px solid black">5</td>
            <td style="border: 1px solid black">6</td>
            <td style="border: 1px solid black">7</td>
            <td style="border: 1px solid black; background: green; color: white;">0</td>
        </tr>
        <tr>
            <td style="background: white; border: none;">0</td>
            <td style="background: white; border: none;">1</td>
            <td style="background: white; border: none;">2</td>
            <td style="background: white; border: none;">3</td>
            <td style="background: white; border: none;">4</td>
            <td style="background: white; border: none;">5</td>
            <td style="background: white; border: none;">6</td>
            <td style="background: white; font-weight: bold; border: none; color: green">7</td>
        </tr>
    </tbody>
</table>

In [11]:
resizableIntArray

OccupiedElements,ArrayState
7,"[ 1, 2, 3, 4, 5, 6, 7, 0 ]"


In [12]:
resizableIntArray.Remove(6);

In [13]:
resizableIntArray


OccupiedElements,ArrayState
6,"[ 1, 2, 3, 4, 5, 7, 0, 0 ]"


<br>

One last thing, let's clear the whole array and return it to it's default state:

In [14]:
resizableIntArray.Clear();

In [15]:
resizableIntArray

OccupiedElements,ArrayState
0,"[ 0, 0, 0, 0 ]"
